In [0]:
%run ../Databricks-Certified-Data-Engineer-Associate/Includes/Copy-Datasets

In [0]:
files = dbutils.fs.ls(f"{dataset_bookstore}/orders-raw")
display(files)

In [0]:
(spark.readStream
    .format("cloudFiles")
    .option("cloudFiles.format", "parquet")
    .option("cloudFiles.schemaLocation", "dbfs:/mnt/chekpoints122/schemaLocation")
    .load(f"{dataset_bookstore}/orders-raw")
    .createOrReplaceTempView("orders_raw_tmp1")
)

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW orders_tmp1 AS
( SELECT *,current_timestamp() as date_created, input_file_name() as source_file
from orders_raw_tmp1

)

In [0]:
%sql
SELECT * FROM orders_tmp1
limit 10

In [0]:
(spark.table("orders_tmp1")
    .writeStream
    .format("delta")
    .option("checkpointLocation", "dbfs:/mnt/chekpoints622/orders_bronze")
    .outputMode("append")
    .table("orders_bronze")
)

In [0]:
%sql
select count(*) from orders_bronze

In [0]:
%sql
SELECT * FROM customers
limit 10

In [0]:
files = dbutils.fs.ls(f"{dataset_bookstore}/customers-json")
display(files)


In [0]:
(
    spark.read
    .format("json")
    .load(f"{dataset_bookstore}/customers-json")
    .createOrReplaceTempView("customer_lookup")

)

In [0]:
# %sql
# -- Here , creating two columns in customers_tmp1
# CREATE OR REPLACE TEMP VIEW customers_lookup AS
# ( SELECT *, current_timestamp() as date_created, input_file_name() as source_file
# FROM customers_tmp1
# )

In [0]:
#before doing any transofrmation on the pipeline, we create a temp view with stream because we want our changes
# to happen in the temp views

(spark.readStream
  .table("orders_bronze")
  .createOrReplaceTempView("orders_bronze_tmp1")
)




In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW ordered_enriched_tmp1 AS
( SELECT order_id, quantity, o.customer_id, c.profile:first_name as f_name, c.profile:last_name as l_name,
    cast(from_unixtime(order_timestamp, "yyyy-MM-dd HH:mm:ss") as timestamp) order_timestamp, books

FROM orders_bronze_tmp1 o
INNER JOIN customer_lookup c
ON o.customer_id = c.customer_id
WHERE quantity > 0
)

In [0]:
%sql
select * from ordered_enriched_tmp1

In [0]:
(spark.table("ordered_enriched_tmp1")
.writeStream
.format("delta")
.option("checkpointLocation", "dbfs:/mnt/chekpoints22/orders_bronze_tmp1")
.outputMode("append")
.table("orders_silver1")
)

In [0]:
%sql
select count(*) from orders_silver1

In [0]:
load_new_data()

In [0]:
# for s in spark.streams.active:
#   print(f"stopped stream: {s.name}")
#   s.stop()
#   s.awaitTermination()